In [1]:
from IPython.display import clear_output

In [2]:
!pip install -U tensorly
clear_output()
import tensorly as tl

In [3]:
!pip install neptune-client
clear_output()
import neptune
project = neptune.init(
    api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiNDc2YjY1N2MtODZiZC00NjMyLWJlN2MtZGJlNWUyOTcxMmQ1In0=',
    project_qualified_name='nazya/cp-l2'
)

### DATA

In [4]:
# import numpy as np
from tqdm.notebook import tqdm
import sys
from sklearn import preprocessing
from matplotlib import pyplot as plt
import random
import time

import autograd.numpy as np
from autograd import grad

def generate_3d_tensor(sizes = 30, rank = 20, mu = 1e-7):
    if type(sizes) == int:
        I,J,K = sizes, sizes, sizes
    else:
        I,J,K = sizes

    a = preprocessing.normalize(np.random.randn(I, rank), norm='l2')
    b = preprocessing.normalize(np.random.randn(J, rank), norm='l2')
    c = preprocessing.normalize(np.random.randn(K, rank), norm='l2')
    N = np.random.randn(I,J,K)
    
    tensor = cp_to_tensor((a,b,c))
    tensor += mu*np.linalg.norm(tensor)/np.linalg.norm(N)*N

    return tensor
    # return tensor, a,b,c

def cp_to_tensor(abc):
    a,b,c = abc
    return np.einsum('ip,jp,kp->ijk', a, b, c)

def RSE(tensor_hat, tensor):
    return np.linalg.norm(tensor_hat - tensor)/np.linalg.norm(tensor)

In [5]:
def scale(factors):
    tensor_norm=1
    for mode in range(factors.shape[0]):
        block_norm = tl.norm(factors[mode])
        factors[mode] /= block_norm
        tensor_norm *= block_norm
    return factors * (tensor_norm**(1/factors.shape[0]))

def warm(factors, rho):
    mask = np.ones(factors.shape[0],dtype=bool)
    eye = np.eye(factors.shape[-1])
    for mode in range(factors.shape[0]):
        mask[mode]=False
        inp  = tl.tenalg.khatri_rao(factors[mask])
        tar = tl.unfold(tensor, mode=mode).T
        factors[mode] = (np.linalg.solve(inp.T @ inp + rho*eye, inp.T @ tar)).T
        mask[mode]=True
    return factors


def generate_starting_point(tensor, rank, rho):
    a = preprocessing.normalize(np.random.random((tensor.shape[0], rank)), norm='l2')
    b = preprocessing.normalize(np.random.random((tensor.shape[1], rank)), norm='l2')
    c = preprocessing.normalize(np.random.random((tensor.shape[2], rank)), norm='l2')
    factors = warm(np.array([a,b,c]), rho)
    return scale(factors)

### ALG

In [6]:
def f(factors, tensor, rho):
    return 0.5*tl.norm(tl.cp_to_tensor((None, factors)) - tensor)**2 + 0.5*rho * tl.norm(factors)**2

In [7]:
def am(factors, tensor, rank, rho, max_time=60):
    tensor_hat  = tl.cp_to_tensor((None, factors))  
    neptune.log_metric('RSE (i)', x=0, y=RSE(tensor_hat, tensor))
    neptune.log_metric('RSE', y=RSE(tensor_hat, tensor), x=0)  
    
    t=0
    start_time = time.time()
    mask = np.ones(factors.shape[0],dtype=bool)
    eye = np.eye(factors.shape[-1])
    while True:
        for mode in range(factors.shape[0]):
            mask[mode]=False
            inp  = tl.tenalg.khatri_rao(factors[mask])
            tar = tl.unfold(tensor, mode=mode).T
            factors[mode] = (np.linalg.solve(inp.T @ inp + rho*eye, inp.T @ tar)).T
            mask[mode]=True
        t-=-3

        if t % 30 == 0:
            stop_time = time.time()
            tensor_hat  = tl.cp_to_tensor((None, factors))
            logging_time = stop_time - start_time
            neptune.log_metric('RSE (i)', x=t, y=RSE(tensor_hat, tensor))
            neptune.log_metric('RSE', y=RSE(tensor_hat, tensor), x=logging_time)  
            if logging_time > max_time:
                return logging_time
            start_time += time.time() - stop_time

In [8]:

y=factors
grad_f_y = np.zeros_like(y)
mask = np.ones(grad_f_y.shape[0],dtype=bool)
X, Y = [], []


NameError: ignored

In [ ]:
%%timeit
for j in range(grad_f_y.shape[0]):    
    mask[j]=False
    inp = tl.tenalg.khatri_rao(y[mask])
    tar = tl.unfold(tensor, mode=j).T
    X.append(rho*eye + inp.T @ inp)
    Y.append(inp.T @ tar)
    grad_f_y[j] = (X[-1]@y[j].T - Y[-1]).T
    mask[j]=True

In [ ]:
dim = 30
rank = 10
noise = 1e-2
rho = noise/10
seed = 0


tensor = generate_3d_tensor(dim, rank, noise)
factors = generate_starting_point(tensor, rank, rho)
mask = np.ones(factors.shape[0],dtype=bool)
eye = np.eye(factors.shape[-1])
mode = 0
mask[mode]=False
inp  = tl.tenalg.khatri_rao(factors[mask])
tar = tl.unfold(tensor, mode=mode).T

%timeit np.linalg.solve(inp.T @ inp + rho*eye, inp.T @ tar).T

# experiments

In [9]:

def check_exp(name, params):
        succExperiments =  project.get_experiments(tag=['finished_successfully', name])
        for exp in succExperiments:
            if exp.get_system_properties()['name'] == name and exp.get_parameters()==params:
                return True
        return False

In [11]:
from tensorly.decomposition._cp import initialize_cp
max_time = 10
n_exp = 7

dim = 30
rank = 10
noise = 1e-2
rho = noise/10
seed = 0


init='svd'
svd='numpy_svd'
orthogonalise = False
normalize_factors = False
random_state=False

name = 'AM-seq'
params={'dim' : dim,
        'rank' : rank,
        'noise' : noise,
        'rho' : rho,
        'seed' : seed}


params_list = []
for seed in range(n_exp):
    params['seed']=seed
    params_list.append(params.copy())


for params in params_list:

    if check_exp(name, params):
        continue
        
    neptune.create_experiment(name=name, params=params)

    random.seed(params['seed'])
    np.random.seed(params['seed'])

    tensor = generate_3d_tensor(dim, rank, noise)
    # _, factors = initialize_cp(tensor, rank, init=init, svd=svd,
    #                              random_state=random_state,
    #                              normalize_factors=normalize_factors)
    # factors = np.array(factors)
    factors = generate_starting_point(tensor, rank, rho)
    t = am(factors, tensor, rank, rho, max_time)
    print(t)

    neptune.append_tags(['finished_successfully', name])
    neptune.stop()

Info (NVML): Driver Not Loaded. GPU usage metrics may not be reported. For more information, see https://docs-legacy.neptune.ai/logging-and-managing-experiment-results/logging-experiment-data.html#hardware-consumption 


https://ui.neptune.ai/nazya/cp-l2/e/CPL2-1171
10.008270978927612
https://ui.neptune.ai/nazya/cp-l2/e/CPL2-1172
10.01019287109375
